# Training Models on Colab

Install TensorFlow and Numpy

In [2]:
!pip install --upgrade pip
!pip install --upgrade protobuf 

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)
Requirement already up-to-date: protobuf in /usr/local/lib/python3.6/dist-packages (3.13.0)


In [3]:
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


Check GPU status

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.6 GB  | Proc size: 466.5 MB
GPU RAM Free: 14968MB | Used: 111MB | Util   1% | Total 15079MB


Mount Google Drive folder

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

# change to working tensorflow directory on the drive
%cd '/content/gdrive/My Drive/models/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AH8m6u36LTpqtBUwBHuvZu054q5eWcZpq4yRVXWQCTBwZS3B7BGlmo
Mounted at /content/gdrive
/content/gdrive/My Drive/models


Install protobuf and compile, install setup.py

In [7]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/gdrive/My Drive/models/research/
!protoc object_detection/protos/*.proto --python_out=.


!python setup.py build
!python setup.py install



Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,791 kB of archives.
After this operation, 7,807 kB of additional disk space will be used.
Get:1 htt

In [8]:

import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/models/research/slim'

Check remaining GPU time

In [9]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.847165993054707


Start training

In [10]:
!pip install tf_slim
%cd /content/gdrive/My Drive/models/research/object_detection
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/models/research/:/content/gdrive/My Drive/Tensorflow/models/research/slim'

!python train.py --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config --logtostderr

Streaming output truncated to the last 5000 lines.
I0911 14:44:37.047047 140420409354112 learning.py:512] global step 863: loss = 1.4435 (0.362 sec/step)
INFO:tensorflow:global step 864: loss = 1.5900 (0.372 sec/step)
I0911 14:44:37.420629 140420409354112 learning.py:512] global step 864: loss = 1.5900 (0.372 sec/step)
INFO:tensorflow:global step 865: loss = 1.4564 (0.353 sec/step)
I0911 14:44:37.775080 140420409354112 learning.py:512] global step 865: loss = 1.4564 (0.353 sec/step)
INFO:tensorflow:global step 866: loss = 1.3906 (0.337 sec/step)
I0911 14:44:38.113186 140420409354112 learning.py:512] global step 866: loss = 1.3906 (0.337 sec/step)
INFO:tensorflow:global step 867: loss = 1.7624 (0.354 sec/step)
I0911 14:44:38.468777 140420409354112 learning.py:512] global step 867: loss = 1.7624 (0.354 sec/step)
INFO:tensorflow:global step 868: loss = 1.3892 (0.335 sec/step)
I0911 14:44:38.805685 140420409354112 learning.py:512] global step 868: loss = 1.3892 (0.335 sec/step)
INFO:tensor

Export inference graph

In [13]:
#  .ckpt needs to be updated every time to match last .ckpt generated
#  .config needs to be updated when changing model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-3227 --output_directory new_graph

Instructions for updating:
Please use `layer.__call__` method instead.
W0911 15:00:43.868282 139883967928192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0911 15:00:45.345384 139883967928192 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0911 15:00:45.383080 139883967928192 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0911 15:00:45.424826 139883967928192 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

Zip file in Google Drive

In [14]:
!zip -r model_graph.zip new_graph

updating: new_graph/ (stored 0%)
  adding: new_graph/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: new_graph/model.ckpt.index (deflated 68%)
  adding: new_graph/checkpoint (deflated 42%)
  adding: new_graph/model.ckpt.meta (deflated 93%)
  adding: new_graph/frozen_inference_graph.pb (deflated 9%)
  adding: new_graph/saved_model/ (stored 0%)
  adding: new_graph/saved_model/variables/ (stored 0%)
  adding: new_graph/saved_model/saved_model.pb (deflated 9%)
  adding: new_graph/pipeline.config (deflated 69%)
